# 컬럼 탐색

- fraud_YN : (B)
    보험 사기 True/False

- car_model 차종
    - ('경형','소형','소형SUV') then 1
    - ('준중형','준중형SUV','중형') then 2
    - ('대형','승합','준대형','중형SUV') then 3
    - ('수입') then 4
    - ('EV','RV') then 5 else 5
- sharing_type:
    - "왕복" then 0
    - else 1 부름 / 쏘카 등등
- age_group 연령대
    - like "%21%" then 1 : 21이 포함되면 1이라는데 그룹핑이 이미 되있는지 불분명
    - like "%23%" then 2
    - like "%27%" then 3
    - like "%31%" then 4
    - like "%41%" then 5
- has_previous_accident 누적 사고 유무
    - 있으면 1
- cumulative_use_count 누적 대여 횟수
    - 1 then 1
    - (2,3,4,5) then 2
    - (6,7,8,9,10) then 3
    - 10 then 4
- b2b 법인이용 (별도 등록)
    - "%P%" then 0 : 개인
    - "%C%" then 1 : 법인
    - "%M%" then 2 : 법인 구성원
    - null then -1
- accident_ratio 과실율
    - ifnull -1
- pf_type 보험료 타입 선택 (PF5, PF30, PF70)
    - like "%PF5%" then 1  : 자기부담금 5만
    - like "%PF30%" then 2  : 자기부담금 30만
    - like "%PF70%" then 3  : 자기부담금 70만
    - else 0
- socarpass 
    - 쏘카패스 유/무
    - 월 x만원 내고 차 대여 비용 할인 받는 서비스 (주행비는 별도)
    - 여러 모델 있음
- socarsave 
    - 쏘카세이브 유/무
    - 3년 이상 지난 차량 싸게 제공 서비스
- start_hour 이용시작시간
    - (5,6,7) then 3
    - (8,9,10) then 4
    - (11,12,13) then 5
    - (14,15,16) then 6
    - (0,1,2,3,4,21,22,23) then 1
    - (17,18,19,20) then 2
    - else -1
- duration 대여기간
    - (2,3,4,5) then 1
    - (6,7,8,9) then 2
    - <=36 then 3
    - > 36 then 4
    - (0,1) then 5
- accident_hour 사고시각 (서울시 기준)
    - (0,1,2,3,4,21,22,23) then 1
    - (17,18,19,20) then 2
    - (5,6,7) then 3
    - (8,9,10) then 4
    - (11,12,13) then 5
    - (14,15,16) then 6
    - else -1
- repair_cost 차량수리비용
    - flaot
- insure_cost 보험손해비용
    - int
- accident_location 사고위치
    - 0주차장 
    - 1일반도로 
    - 2이면도로 
    - 3고속도로 
    - 4쏘카존 
    - 5확인불가
- car_part1 전면 손상
    - 유/무
- car_part2 후면 손상
    - 유/무
- repair_cnt 수리 부위 갯수 int64
    - 
- acc_type1 사고타입(전면충돌, 차대인, 자전거, 알수없음 등등)
    - "차대차" then 1
    - "차대인" then 2
    - "차량단독" then 3
    - "차대자전거" then 4
    - else 0
- insurance_site_aid_YN 보험사 현장출동 유무
    - "%Y%" then 1
    - "%N%" then 2
    - else 0
- police_site_aid_YN 
    - "%신고%" then 1
    - "%미신고%" then 2
    - else 0
- total_prsn_cnt 탑승인원
    - "5 이상" then 5
    - "미확인" then -1, else -1
- test_set
    - created_at >= '2020-02-01' and created_at < '2020-07-15' then 1
    - else 0

In [2]:
df = pd.read_csv('./datas/insurance_fraud_detect_data.csv')
df.head()

,fraud_YN,car_model,sharing_type,age_group,has_previous_accident,cumulative_use_count,b2b,accident_ratio,pf_type,socarpass,...,insure_cost,accident_location,car_part1,car_part2,repair_cnt,acc_type1,insurance_site_aid_YN,police_site_aid_YN,total_prsn_cnt,test_set
0,0,2,1,3,0,2,0,100,2,0,...,0,1,0,0,1,0,0,0,-1,0
1,0,1,0,4,0,4,0,0,3,1,...,0,1,1,1,2,1,1,1,0,1
2,0,1,0,4,1,4,0,100,3,1,...,12925040,3,1,0,2,1,1,1,0,1
3,0,3,1,3,0,4,0,0,2,0,...,0,3,1,0,1,0,0,0,-1,0
4,0,1,0,1,0,1,0,100,1,0,...,1209410,1,0,0,2,0,0,0,-1,0


In [10]:
pd.options.display.max_columns = None

In [11]:
df.describe()

,fraud_YN,car_model,sharing_type,age_group,has_previous_accident,cumulative_use_count,b2b,accident_ratio,pf_type,socarpass,socarsave,start_hour,duration,accident_hour,repair_cost,insure_cost,accident_location,car_part1,car_part2,repair_cnt,acc_type1,insurance_site_aid_YN,police_site_aid_YN,total_prsn_cnt,test_set
count,16000.000000,16000.000000,16000.000000,16000.000000,16000.000000,16000.000000,16000.000000,16000.000000,16000.000000,16000.000000,16000.000000,16000.000000,16000.000000,16000.000000,1.600000e+04,1.600000e+04,16000.000000,16000.000000,16000.000000,16000.000000,16000.000000,16000.000000,16000.000000,16000.000000,16000.000000
mean,0.002563,1.695125,0.140563,2.484625,0.085312,2.554875,0.013125,89.061562,1.757125,0.162875,0.080062,3.729563,2.254313,3.356438,2.850116e+05,8.594367e+05,1.288625,0.329313,0.173250,1.358375,0.299312,0.359500,0.292000,-0.461875,0.195062
std,0.050558,0.760156,0.347581,1.165428,0.279355,1.162841,0.161491,29.688451,0.757247,0.369263,0.271398,1.716278,1.149878,1.984537,5.172140e+05,3.094520e+06,1.327377,0.469978,0.378475,0.908085,0.487842,0.604076,0.454697,0.995670,0.396261
min,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,-1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000
25%,0.000000,1.000000,0.000000,2.000000,0.000000,2.000000,0.000000,100.000000,1.000000,0.000000,0.000000,2.000000,1.000000,2.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-1.000000,0.000000
50%,0.000000,2.000000,0.000000,2.000000,0.000000,2.000000,0.000000,100.000000,2.000000,0.000000,0.000000,4.000000,2.000000,2.000000,1.881183e+05,0.000000e+00,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-1.000000,0.000000
75%,0.000000,2.000000,0.000000,3.000000,0.000000,4.000000,0.000000,100.000000,2.000000,0.000000,0.000000,5.000000,3.000000,5.000000,3.633458e+05,5.651200e+05,2.000000,1.000000,0.000000,2.000000,1.000000,1.000000,1.000000,0.000000,0.000000
max,1.000000,5.000000,1.000000,5.000000,1.000000,4.000000,2.000000,100.000000,3.000000,1.000000,1.000000,6.000000,5.000000,6.000000,9.713774e+06,9.471435e+07,5.000000,1.000000,1.000000,37.000000,3.000000,2.000000,1.000000,5.000000,1.000000


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   fraud_YN               16000 non-null  int64  
 1   car_model              16000 non-null  int64  
 2   sharing_type           16000 non-null  int64  
 3   age_group              16000 non-null  int64  
 4   has_previous_accident  16000 non-null  int64  
 5   cumulative_use_count   16000 non-null  int64  
 6   b2b                    16000 non-null  int64  
 7   accident_ratio         16000 non-null  int64  
 8   pf_type                16000 non-null  int64  
 9   socarpass              16000 non-null  int64  
 10  socarsave              16000 non-null  int64  
 11  start_hour             16000 non-null  int64  
 12  duration               16000 non-null  int64  
 13  accident_hour          16000 non-null  int64  
 14  repair_cost            16000 non-null  float64
 15  in

In [3]:
# 사기인가?
df.fraud_YN.unique()

array([0, 1])

In [4]:
# 차종?
df.car_model.unique()

array([2, 1, 3, 5, 4])

In [5]:
df.sharing_type.unique()

array([1, 0])

In [6]:
df.age_group.unique()

array([3, 4, 1, 2, 5])

In [7]:
df.has_previous_accident.unique()

array([0, 1])

In [8]:
df.cumulative_use_count.unique()

array([2, 4, 1, 3])

In [9]:
df.b2b.unique()

array([0, 2])

In [12]:
df.total_prsn_cnt.unique()

array([-1,  0,  2,  1,  3,  4,  5])

In [13]:
df.acc_type1.unique()

array([0, 1, 2, 3])